In [89]:
import math
import time
import random as rd
from collections import Counter
import threading

deck = []
hand_output = []
fastmode = True
hard_mode = False

# Card
class Card:
        def __init__(self, rank, suit):
            self.rank = rank
            self.suit = suit
            
        def __eq__(self, other):
            return isinstance(other, Card) and self.rank == other.rank and self.suit == other.suit
        
        def __hash__(self):
            return hash((self.rank, self.suit))
            
        def __repr__(self):
            return f"Card('{self.rank}', '{self.suit}')"
            
        def __str__(self):
            return f"{self.rank} of {self.suit}"



# Rank and suits as well as conversion from player input to proper ranks
ranks = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
rank_order = {'1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'J':11 , 'Q':12, 'K':13, 'A':14}
suits = ["Spades", "Diamonds", "Clubs", "Hearts"]
rank_input_map = {'Ace' : 'A', 'A': 'A', 'Two': '2', '2': '2', 'Three': '3', '3': '3', 'Four': '4', '4': '4', 'Five': '5', '5': '5', 'Six': '6', '6':'6',
'Seven': '7', '7': '7', 'Eight': '8', '8': '8', 'Nine': '9', '9': '9', 'Ten': '10', '10': '10', 'Jack': 'J', 'J': 'J', 'Queen': 'Q', 'Q': 'Q',
'King':'K', 'K': 'K'}

# Creates all 52 cards
def init_deck():
    global deck
    deck = [Card(rank, suit) for rank in ranks for suit in suits]

# Create the deck and the first 5 community cards
init_deck()
community = card_create(5)

# Creates 'x' cards
def card_create(x):
    # Form deck
    global deck
    
    if len(deck) < x:
        raise ValueError("Not enough cards in deck to draw.")
        
    temp = rd.sample(deck, x) 
    for card in temp:
        deck.remove(card)
    return temp

# Returns hand_output to cards
def reset_cards(cards, hand_output):
    for card in hand_output:
        cards.append(card)
    hand_output.clear()

# Returns sorted cards (no duplicates)
def sort_cards(cards):
    unique_cards = []
    seen_rank = set()
    # Sorts cards by their rank
    sorted_cards = sorted(cards, key=lambda card: rank_order[str(card.rank)])

    # Checks if a card with that rank has had its rank noted yet, if not, adds it
    for card in sorted_cards:
        rank = rank_order[str(card.rank)]
        if rank not in seen_rank:
            unique_cards.append(card)
            seen_rank.add(rank)

    return unique_cards
    
# Returns sorted cards in reverse order (no duplicates)
def reverse_sort(cards):
    unique_cards = []
    seen_rank = set()
    # Sorts cards by their rank
    sorted_cards = sorted(cards, key=lambda card: rank_order[str(card.rank)], reverse = True)

    # Checks if a card with that rank has had its rank noted yet, if not, adds it
    for card in sorted_cards:
        rank = rank_order[str(card.rank)]
        if rank not in seen_rank:
            unique_cards.append(card)
            seen_rank.add(rank)   

    return unique_cards
# Returns all cards sorted 
def sort_all_cards(cards):
    all_cards = []
    sorted_cards = sorted(cards, key=lambda card: rank_order[str(card.rank)])
    all_cards.extend(sorted_cards)
         
    return all_cards


# Returns all cards sorted in reverse order
def reverse_sort_all_cards(cards):
    all_cards = []
    sorted_cards = sorted(cards, key=lambda card: rank_order[str(card.rank)], reverse = True)
    all_cards.extend(sorted_cards)
         
    return all_cards
    
# Returns a 'strength' and 'tiebreak' attribute to each hand as well as a hierachy for order of searching
def eval_hand(cards):
    hand_output = []
    sorted_cards = sort_cards(cards)
    
    straight_flush = detect_straight_flush(cards, [])
    if straight_flush:
        _, hand_output = straight_flush
        high_card_rank = rank_order[str(hand_output[-1].rank)]
        name = "straight flush"
        return (8, high_card_rank, hand_output, name)
        
    quads = detect_quads(cards, [])
    if quads:
        temp, hand_output = quads
        temp = sort_all_cards(temp)
        kicker_rank = rank_order[str(temp[-1].rank)]
        name = "quads"
        return (7, kicker_rank, hand_output, name)

    full_house = detect_full_house(cards, [])
    if full_house:
        _, hand_output = full_house
        full_of = rank_order[str(hand_output[0].rank)]
        house = rank_order[str(hand_output[-1].rank)]
        name = "full house"
        return (6, [full_of, house], hand_output, name)
        
    flush = detect_flush(cards, [])
    if flush:
        _, hand_output = flush
        high_card_rank = rank_order[str(hand_output[-1].rank)]
        name = "flush"
        return (5, high_card_rank, hand_output, name)
    
    straight = detect_straight(cards, [])
    if straight:
        hand_output = straight
        high_card_rank = rank_order[str(hand_output[-1].rank)] 
        name = "straight"
        return (4, high_card_rank, hand_output, name)
    
    set_ = detect_set(cards, hand_output)
    if set_:
        _, hand_output = set_
        rank = rank_order[str(hand_output[0].rank)]
        name = "set"
        return (3, rank, hand_output, name)
    
    two_pair = detect_two_pair(cards, hand_output)
    if two_pair:
        _, hand_output = two_pair
        rank1 = rank_order[str(hand_output[0].rank)]
        rank2 = rank_order[str(hand_output[-1].rank)]
        name = "two pair"
        return (2, [rank1, rank2], hand_output, name)
   
    pair = detect_pair(cards)
    if pair:
        _, hand_output = pair
        rank = rank_order[str(hand_output[0].rank)]
        name = "pair"
        return (1, rank, hand_output, name)
    else:
        kicker_rank = rank_order[str(sorted_cards[-1].rank)]
        high_card = sorted_cards[-1]
        hand_output = [high_card]
        name = "high card"
        return (0, kicker_rank, hand_output, name)

# Compares the strengths of two hands and outputs the winning hand
def compare_hands(eval_hand1, eval_hand2):

    # First and last comparison of hands
    if eval_hand1 is None:
        return eval_hand2
    if eval_hand2 is None:
        return eval_hand1

    # Unpacking values from eval_hand
    strength1, tiebreak1, hand_output1, _ = eval_hand1
    strength2, tiebreak2, hand_output2, _ = eval_hand2

    # List of hand strength with no additional rules needed
    normalbreak = [0, 1, 3, 4, 8] 
    # Directly compares hands with no tiebreakers
    if strength1 > strength2:
        return eval_hand1
    if strength2 > strength1:
        return eval_hand2

    if tiebreak1 is None:
        return eval_hand2
    if tiebreak2 is None:
        return eval_hand1

    tiebreakfavor1 = tiebreak1 > tiebreak2
    tiebreakfavor2 = tiebreak1 < tiebreak2

    # Compares the tiebreakers of certain same hands
    if (strength1 == strength2):
        if strength1 in normalbreak:
            if tiebreakfavor1:
                return eval_hand1 
            if tiebreakfavor2:
                return eval_hand2
                
        if strength1 == 2 or 6:
            if tiebreakfavor1:
                return eval_hand1
            if tiebreakfavor2:
                return eval_hand2

    if strength1 == 7:
        if tiebreakfavor1:
            return eval_hand1 
        elif tiebreakfavor2:
            return eval_hand2
        
    # Compares each card of the flush from highest to lowest and compares against the other       
    if strength1 == 5:
        sort_hand1 = reverse_sort_all_cards(hand_output1)
        sort_hand2 = reverse_sort_all_cards(hand_output2)
        
        for i in range(5):
            rank1 = rank_order[str(sort_hand1[i].rank)]
            rank2 = rank_order[str(sort_hand2[i].rank)]
        
            if rank1 > rank2:
                return eval_hand1
            elif rank2 > rank1:
                return eval_hand2
            
    return eval_hand1

        
# Determines best hand
def best_hand(cards, deck):
    best_hand = None
    best_two_cards = None
    remaining_cards = [card for card in deck if card not in cards]

    # Cycles through every possible two card hand with remaining cards
    for i in range(len(remaining_cards)):
        for j in range(i+1, len(remaining_cards)):
            two_cards = [remaining_cards[i], remaining_cards[j]]
            full_hand = cards + two_cards

            # Gets a strength for current seven card hand
            hand_strength = eval_hand(full_hand)

            # Returns the current hand during first test when no hand has been checked yet
            if best_hand is None:
                best_two_cards = two_cards
                best_hand = hand_strength
            else:
                # Best hand is the stronger of the last and current
                best_hand = compare_hands(hand_strength, best_hand)
                # If best hand changed, also update the two cards with that hand
                if compare_hands(hand_strength, best_hand) == hand_strength:
                    best_two_cards = two_cards
                    best_hand = hand_strength
           
    return (best_hand, best_two_cards)
    
# Finds two of a kind
def detect_pair(cards):

    # Creates list of all ranks, counts their frequency, if exactly 2, it is a pair
    ranks = [card.rank for card in cards]
    count_ranks = Counter(ranks)
    # Creates a list of ranks that appear twice  
    pair_ranks = [rank for rank, count in count_ranks.items() if count == 2]
    
    if not pair_ranks:
        return False
        
    # Sorts ranks which appear twice by descending rank and then picks highest rank
    pair_ranks.sort(key=lambda card: rank_order[str(card)], reverse = True)
    target_rank = pair_ranks[0]

    # Find all cards in input cards with rank in pair_ranks
    pair_cards = [card for card in cards if card.rank == target_rank]

    remaining_cards = [card for card in cards if card.rank != target_rank]
            
    return (remaining_cards, pair_cards)
    
        
# Finds two, two of a kinds
def detect_two_pair(cards, hand_output):
    unique_cards = reverse_sort_all_cards(cards)
    result = detect_pair(unique_cards)

    # Checks if there are any pairs
    if not result:
        return False

    # Finds the rank of the highest pair from result
    temp, hand_output = result
    found_rank = hand_output[0].rank
    
    for i, card in enumerate(temp[:]):
        # Compare with every card after itself
        for j, other_card in enumerate(temp[i+1:], start = i+1):
            # Looks in the remaining cards for any pairs which do not share the rank from the first pair
            if card.rank == other_card.rank and card.rank != found_rank:
                hand_output.append(card)
                hand_output.append(other_card)
                temp.remove(card)
                temp.remove(other_card)
                return (temp, hand_output)
    else:
        return False

    
# Finds three of a kind
def detect_set(cards, hand_output):
    # Creates list of all ranks, counts their frequency, if exactly 3, it is a set
    ranks = [card.rank for card in cards]
    count_ranks = Counter(ranks)
    triple_ranks = [rank for rank, count in count_ranks.items() if count == 3]
    
    if not triple_ranks:
        return False
        
    # Sorts ranks which appear thrice by descending rank 
    triple_ranks.sort(key=lambda card: rank_order[str(card)], reverse = True)
    target_rank = triple_ranks[0]

    # Find the cards from the input cards with the desired rank 
    for card in cards[:]:
        if card.rank == target_rank:
            cards.remove(card)
            hand_output.append(card)
            
    return (cards, hand_output)
    
            
# Finds four of a kind
def detect_quads(cards, hand_output):
    # Creates list of all ranks, counts their frequency, if exactly 4, it is quads
    ranks = [card.rank for card in cards]
    count_ranks = Counter(ranks)
    quad_ranks = [rank for rank, count in count_ranks.items() if count == 4]

    if not quad_ranks:
        return False
    # Sorts ranks which appear four times by descending rank 
    quad_ranks.sort(key=lambda card: rank_order[str(card)], reverse = True)
    target_rank = quad_ranks[0]

    # Find the cards from the input cards with the desired rank 
    for card in cards[:]:
        if card.rank == target_rank:
            cards.remove(card)
            hand_output.append(card)

    return (cards, hand_output)
            
# Finds a full house (a unique set and pair)
def detect_full_house(cards, hand_output):
    reset_cards(cards, hand_output)
    unique_cards = reverse_sort_all_cards(cards)
    result = detect_set(unique_cards, hand_output)
    
    # Checks if there was a set, if not, no full house
    if result == False:
        return False
    else:
        # Set is saved
        unique_cards, hand_output = result
        # Remaining cards are used to check for a pair and is joined with set
        result = detect_pair(unique_cards) 
        if result == False:
            return False
        else:
            unique_cards , result_new = result
            hand_output.extend(result_new)
            
            return (unique_cards, hand_output)

# Finds a flush (five of the same suit)
# FYI: will fail if more than one suit is a flush and instead outputs first 5 cards in those suits
def detect_flush(cards, hand_output, returnall = False):
    reset_cards(cards, hand_output)
    unique_cards = sort_all_cards(cards)
    # Array of all suits
    suits = [card.suit for card in unique_cards]
    # Dictionary of suits and their count
    count_suit = Counter(suits)
    # Array of suits which appear five times or more
    target_suit = [suit for suit, count in count_suit.items() if count >= 5]

    if not target_suit:
        return False
        
    hold = []
    # Find the cards with suits which appear five times or more
    for card in unique_cards[:]:
        if card.suit in target_suit:
            hold.append(card)
            unique_cards.remove(card)
    # Allows for the option to return all the cards of that suit if there are more than 5, useful for finding highest straight flush
    if returnall == True:
        hand_output.extend(hold)
    else:
        hand_output.extend(hold[-5:])
        
    if not hand_output:
        return False
    # Reverse sorts all cards before outputting for consistency
    hand_output = reverse_sort_all_cards(hand_output)
    
    return (unique_cards, hand_output)



# Finds a straight (five cards of consecutive rank)
def detect_straight(cards, hand_output):
    unique_cards = reverse_sort_all_cards(cards)
    
    # List of sorted ranks
    ranks = sorted({rank_order[str(card.rank)] for card in unique_cards}, reverse=True)

    # Hard codes ace-low and high exceptions
    if {10, 11, 12, 13, 14}.issubset(ranks):
        straight_ranks = {10, 11, 12, 13, 14}
        is_wheel = False
    elif {14, 2, 3, 4, 5}.issubset(ranks):
        straight_ranks = {14, 2, 3, 4, 5}
        is_wheel = True
    else:
    # Check for straights with 5 unique consecutive ranks
        straight_found = False
        high_card_rank = None
    
        # Iterates backwards and only up to the fifth from last card (four cards from end)
        for i in range(len(ranks) - 4):
            if ranks[i] - ranks[i+4] == 4:
                straight_found = True
                high_card_rank = ranks[i]
                break
                    
        if not straight_found:
            return False

        # Determines the range from the highest card
        straight_ranks = set(range(high_card_rank - 4, high_card_rank + 1))
        is_wheel = (high_card_rank == 5)

    # Finds the cards in the straight ranks
    hand_output = []
    for rank in sorted(straight_ranks):
        for card in unique_cards:
            if rank_order[str(card.rank)] == rank:
                hand_output.append(card)
                break
    
    if len(hand_output) != 5:
            return False
    # Moves the ace to the front if it is an ace-low
    if is_wheel:
        for i, card in enumerate(hand_output):
            if card.rank == "A":
                ace_index = i
                break
                
        ace = hand_output.pop(ace_index)
        hand_output.insert(0, ace)

    
    return hand_output
        
# Finds a straight flush (five cards of the same suit and in consecutive order)
def detect_straight_flush(cards, hand_output):
    # Check if a flush exists and assign those cards to tuple 'result' else is bool 'false'
    result = detect_flush(cards, [], returnall=True)
    
    if not result:
        return False

    # Unpack just the cards in the flush into 'result_new'
    _ , result_new = result

    # Checks for a straight in the five flush cards
    straight = detect_straight(result_new, [])

    if not straight:
        return False
    
    return (cards, straight)    

# Capitalizes text, excluding of
def capitalize_card(text):
    exception = {"of"}
    words = text.lower().split()
    capitalized = [word if word in exception else word.capitalize() for word in words]
    return " ".join(capitalized)

# Sleep function that is reduced in duration when fastmode parameter is true
def conditional_sleep(seconds, fastmode):
    if not fastmode:
        time.sleep(seconds)
    if fastmode:
        time.sleep(seconds/2)

# Runs the entirety of the poker game
def poker_hand_game(cards, hand_output, deck, fastmode):
    conditional_sleep(0.5, fastmode)
    print("Hello there!")
    conditional_sleep(1.5,fastmode)
    print("Would you like to play a game? Y/N")
    conditional_sleep(1, fastmode)
    q1 = input("")

    # Ensures an acceptable answer is given
    while True:
        if q1.lower() == 'n':
            conditional_sleep(1, fastmode)
            print("Alright, see you!")
            return
        elif q1.lower() == 'y':
            conditional_sleep(1, fastmode)
            print("Great! Let's play.")
            break
        else:
            print("Please enter 'Y' or 'N'.")

    conditional_sleep(1, fastmode)
    print("What's your name?")
    conditional_sleep(1, fastmode)
    name = input("")
    print(f"Well hello there, {name}!")
    conditional_sleep(1, fastmode)
    print("Would you like to activate fastmode? (reduces sleep duration between text) Y/N")
    conditional_sleep(1, fastmode)
    q3 = input("")

    # Choice for fastmode, reducing sleep durations
    if q3.lower() == 'y':
        fastmode = True
        print("Fastmode activated!")
        
    else:
        conditional_sleep(.75, fastmode)
        print("No fastmode today!")
        
    conditional_sleep(1.75, fastmode)
    print("Would you like to play on hardmode? (sets a decreasing timer to increase difficulty each round) Y/N")
    q4 = input("")

    # Choice for hardmode, adds a timer decreasing with round progression for additional difficulty
    if q4.lower() == 'y':
        hard_mode = True
        print("Hardmode activated!")
        
    else:
        hard_mode = False
        conditional_sleep(.75, fastmode)
        print("Easy mode then!")
        
    conditional_sleep(1.75, fastmode)
    print(f"Now that we've gotten past that {name}, it's gametime!")
    conditional_sleep(1.75, fastmode)
    print("Let me introduce you to the rules.")
    conditional_sleep(1.75, fastmode)
    print("Five cards will be drawn from the deck and your job will be to figure out what the two cards not on the board would create the best possible hand!")
    conditional_sleep(3.5, fastmode)
    print("If you're correct, you get a point, if you're not, the computer will tell you the correct answer and get a point.")
    conditional_sleep(2, fastmode)
    if hard_mode:
        print("This is hardmode, you need 10 points to win, the AI only needs 3.")
        conditional_sleep(1.5, fastmode)
    else:
        print("First to 10 wins.")
        conditional_sleep(1.5, fastmode)
    input("Got it?")
    conditional_sleep(2.25, fastmode)
    print("Alright, let's get started.")
    conditional_sleep(1.5, fastmode)

    # Runs the code for the game to start
    run_game(community, deck, fastmode, hard_mode)

# Runs the gameplay portion of poker game
def run_game(cards, deck, fastmode, hard_mode):
    player_score = 0
    ai_score = 0
    round_number = 1
    timer_expired = [False]
    timer_stop = [False]

    # Runs code for timer for hardmode
    if hard_mode:
        def countdown(seconds, stop):
            count = 10 - ((((round_number - 6.473702)**3)/41) + 4)
            rounded_count = round(count, 2)
            print(f"You have {rounded_count} seconds")
            if count > 5 and not stop[0]:
                sleep_1 = count - 5
                time.sleep(sleep_1)
                print("5 seconds left!") 
                time.sleep(2)
                print("3 seconds left!")
                time.sleep(3)
                
            elif not stop[0]:
                    sleep_2 = count - 3
                    time.sleep(sleep_2)
                    print("3 seconds left!")
                    time.sleep(3)
                
            if not stop[0]:
                print("Time is up!")
                timer_expired[0] = True

    # Plays the game until someone reaches a score of 10
    while (player_score < 10) and (ai_score < 10):
        print(f"===== Round {round_number} =====")

        init_deck()
        current_deck = deck.copy()
        
        community = card_create(5)

        # Intro text only occurs once
        if round_number == 1:
            print("Here are the cards you'll try and find the best hand with:")
            
        conditional_sleep(1, fastmode)
        for card in community:
            conditional_sleep(1, fastmode)
            print(str(card))
            
        conditional_sleep(1.5, fastmode)    
        print("===================")
        conditional_sleep(2, fastmode)
        
        if round_number == 1:
            print("Submit your answer in the form, 'rank' of 'suit', ex: Two of Spades or 2 of Spades. Varying suit in straights and sets do not change the answer.")
    
        card1 = community[0]
        card2 = community[1]
        conditional_sleep(1.2, fastmode)

        # Timer thread is created if hardmode is on, recreated each round
        if hard_mode:
            timer_stop[0] = False
            timer_expired[0] = False
            timer_thread = threading.Thread(target = countdown, args = (10 - ((((round_number - 5.473702)**3)/41) + 4), timer_stop))
            timer_thread.daemon = True
            timer_thread.start()

        # Query loop for player cards
        while True:  
            first_card = input("What is your first card?") 
            conditional_sleep(1, fastmode)
            second_card = input("What is your second card?")
            conditional_sleep(1, fastmode)
            card1 = capitalize_card(first_card)
            card2 = capitalize_card(second_card)
            card1_list = card1.split()
            card2_list = card2.split()
            condition1 = len(card1_list) != 3 or card1_list[1] != "of" or len(card2_list) != 3 or card2_list[1] != "of"
            
            if timer_expired[0]:
                print("You were too slow, better luck next time!")
                print(f"Score: {player_score}:{ai_score}")
                return

            # Ensures valid input from player
            if condition1:
                print("Invalid response given, try again!")
            else:
                if (card1_list[0] in rank_input_map) and (card2_list[0] in rank_input_map):
                    card1 = Card(rank_input_map[card1_list[0]], card1_list[2])
                    card2 = Card(rank_input_map[card2_list[0]], card2_list[2])
                    condition3 = (card1_list[0] not in rank_input_map or card2_list[0] not in rank_input_map) or (card1_list[2] not in suits or card2_list[2] not in suits)
                    condition2 = (card1 in community) or (card2 in community)
                    condition4 = (card1 == card2)
                    
                    if condition3:
                        print("Invalid response given, try again!")
                        card1_list.clear()
                        card2_list.clear()
                    elif condition2:
                        print("One of your cards was already dealt, try again!")
                        card1_list.clear()
                        card2_list.clear()
                    elif condition4:
                        print("Same cards input. Did you mean to change the suit?")
                        card1_list.clear()
                        card2_list.clear()
                    else:
                        break
                
                else:
                    print("Invalid response given, try again!")
                    card1_list.clear()
                    card2_list.clear()    
           
        round_number += 1
        
        player_copy = community[:]
        player_copy.append(card1)
        player_copy.append(card2)

        # Finds and outputs the hand the player created
        player_answer = eval_hand(player_copy)
        _, _, player_hand_output, player_card_hand = player_answer
        print("You created a", player_card_hand)
        for i in range(len(player_hand_output)):
            print(str(player_hand_output[i]))

        # Find and output the cards the AI found to be best
        temp = best_hand(community, deck)
        ai_answer, ai_cards = temp
        best_hand_strength, _, ai_hand_output, ai_card_hand = ai_answer

        # Differs for grammatical correctness for quads
        if best_hand_strength == 7:
            print("The best hand was", ai_card_hand, "using the", ai_cards[0], "and", ai_cards[1])
        else:
            print("The best hand was a", ai_card_hand, "using the", ai_cards[0], "and", ai_cards[1])

        timer_stop[0] = True

        # Determines if the player is correct or not and displays score
        if check_player(player_hand_output, ai_hand_output, best_hand_strength) == True:
            player_score += 1
            print(f"You win! Score: {player_score}:{ai_score}.")
        else:
            ai_score += 1
            print(f"AI wins! Score: {player_score}:{ai_score}.")
            # Ends game for hardmode if AI gets 3 points
            if hard_mode:
                if ai_score <= 3:
                    print(f"AI only need {3 - ai_score} more points to win")
                else:
                    print(f"Game over! Score: {player_score}:{ai_score}")
                    return 
    # End credit if player wins
    if player_score >= 10:
        if hard_mode:
            print("Wow, didn't know it was possible, congrats!")
            conditional_sleep(1.5, fastmode)
            print("You can rest now.")
        else:
            print("You won the game!")
    else:
        print("AI won the game!")

# Determines if the player answer is correct or not     
def check_player(player_hand_output, ai_hand_output, best_hand_strength):
    rank_suit = [5, 8]
    rank_only = [3, 4, 7]

    if best_hand_strength in rank_suit:
        return player_hand_output == ai_hand_output

    elif best_hand_strength in rank_only:
        player_hand_ranks = sorted({rank_order[str(card.rank)] for card in player_hand_output})
        ai_hand_ranks = sorted({rank_order[str(card.rank)] for card in ai_hand_output})
        return player_hand_ranks == ai_hand_ranks
        
    else:
        return False
    

poker_hand_game(community, [], deck, True)
#run_game(community, deck, True, True)

                        

        
        
    
    
    

'''result = best_hand([Card('3', 'Diamonds'), Card('6', 'Spades'), Card('10', 'Clubs'), Card('K', 'Hearts'), Card('5', 'Diamonds')], deck)

if result:
    temp1, temp2 = result
    score, rank_info, hand_output, name_of_hand = temp1 
    card_output = "\n".join(f"{card.rank} of {card.suit}" for card in hand_output)
    print("Best hand was a ", name_of_hand, ":\n", card_output, sep ='')
    print(f"It used {temp2}")'''

Hello there!
Would you like to play a game? Y/N


 y


Great! Let's play.
What's your name?


 y


Well hello there, y!
Would you like to activate fastmode? (reduces sleep duration between text) Y/N


...

What is your first card? k of spades


Time is up!


What is your second card? k of diamonds


You were too slow, better luck next time!
Score: 2:1


'result = best_hand([Card(\'3\', \'Diamonds\'), Card(\'6\', \'Spades\'), Card(\'10\', \'Clubs\'), Card(\'K\', \'Hearts\'), Card(\'5\', \'Diamonds\')], deck)\n\nif result:\n    temp1, temp2 = result\n    score, rank_info, hand_output, name_of_hand = temp1 \n    card_output = "\n".join(f"{card.rank} of {card.suit}" for card in hand_output)\n    print("Best hand was a ", name_of_hand, ":\n", card_output, sep =\'\')\n    print(f"It used {temp2}")'

In [ ]:
[1,2,3,4,5]